<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2021/examples/blob/main/example-spark/imdb-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB File Processing

This example shows the basic RDD processing.



In [1]:
#Basic installation requires you to setup java. Colab does not have java - so we install.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
#install spark. we are using the one that uses hadoop as the underlying scheduler.
!wget -q https://downloads.apache.org/spark//spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!ls -l


In [5]:
#Provides findspark.init() to make pyspark importable as a regular library.
os.environ["SPARK_HOME"] = "spark-3.1.1-bin-hadoop3.2"
!pip install -q findspark
import findspark
findspark.init()

In [6]:
#The entry point to using Spark SQL is an object called SparkSession. It initiates a Spark Application which all the code for that Session will run on
# to larn more see https://blog.knoldus.com/spark-why-should-we-use-sparksession/
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

# Now the installation is complete. And we will start with processing.



In [7]:
#Lets get the imdb file
!wget -q https://raw.githubusercontent.com/vu-topics-in-big-data-2021/examples/main/example-spark/data/imdb.csv

In [8]:
#this will infer the schema -- for example column names. It creates an RDD
data = spark.read.csv('imdb.csv',inferSchema=True, header=True)

In [9]:
#just shows the basic count of rows and number of columns
data.count(), len(data.columns)

(1000, 12)

In [10]:
#To view a DataFrame, use the .show() method:
data.show(5)

+----+--------------------+--------------------+--------------------+--------------------+--------------------+----+-----------------+------+------+------------------+---------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+----+-----------------+------+------+------------------+---------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|2014|              121|   8.1|757074|            333.13|     76.0|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|2012|              124|     7|485820|            126.46|     65.0|
|   3|               Split|     Horror,Thriller|Three girls are k...|  M. Night Shyamalan|James McAvoy, Any...

In [13]:
#will not truncate and show 5 rows
data.show(5, False)

+----+-----------------------+------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+--------------------------------------------------------------------------+----+-----------------+------+------+------------------+---------+
|Rank|Title                  |Genre                   |Description                                                                                                                                                                                                                   |Director            |Actors                                                                    |Year|Runtime (Minutes)|Rating|Votes |Revenue (Millions)|Metascore|
+----+-----------------------+------------------------+---------------------------------------------------------------

In [14]:
#to see data schema
data.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Runtime (Minutes): string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Votes: string (nullable = true)
 |-- Revenue (Millions): double (nullable = true)
 |-- Metascore: double (nullable = true)



In [15]:
#We can also selectively choose which columns we want to display with the .select() method
data.select("Title","Genre","Director","Year").show(5, truncate=False) #truncate=False parameter that adjusts the size of columns to prevent values from being cut off.

+-----------------------+------------------------+--------------------+----+
|Title                  |Genre                   |Director            |Year|
+-----------------------+------------------------+--------------------+----+
|Guardians of the Galaxy|Action,Adventure,Sci-Fi |James Gunn          |2014|
|Prometheus             |Adventure,Mystery,Sci-Fi|Ridley Scott        |2012|
|Split                  |Horror,Thriller         |M. Night Shyamalan  |2016|
|Sing                   |Animation,Comedy,Family |Christophe Lourdelet|2016|
|Suicide Squad          |Action,Adventure,Fantasy|David Ayer          |2016|
+-----------------------+------------------------+--------------------+----+
only showing top 5 rows



In [16]:
#summary statistics
data.describe(["Runtime (Minutes)","Rating","Title"]).show()

+-------+--------------------+------------------+--------------------+
|summary|   Runtime (Minutes)|            Rating|               Title|
+-------+--------------------+------------------+--------------------+
|  count|                1000|              1000|                1000|
|   mean|  126.65829145728644|15.428728728728741|   635.6666666666666|
| stddev|  160.08961216341075|126.98048279758784|   860.1559548515994|
|    min| teamed up on a j...|               1.9|(500) Days of Summer|
|    max|Taraneh Alidoosti...| Baltasar Kormákur|            Zootopia|
+-------+--------------------+------------------+--------------------+



In [17]:
#we can rename columns
df2 = data.withColumnRenamed("Runtime (Minutes)","runtime").withColumnRenamed("Revenue (Millions)","revenue")
df2.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Votes: string (nullable = true)
 |-- revenue: double (nullable = true)
 |-- Metascore: double (nullable = true)



In [18]:
#group by is a transformation 
# https://backtobazics.com/big-data/spark/apache-spark-groupby-example/
df2.groupBy("Director").count().orderBy("count", ascending=False).show(10)

+------------------+-----+
|          Director|count|
+------------------+-----+
|      Ridley Scott|    8|
|M. Night Shyamalan|    6|
|       David Yates|    6|
|       Michael Bay|    6|
|Paul W.S. Anderson|    6|
|       Danny Boyle|    5|
|       Zack Snyder|    5|
|       J.J. Abrams|    5|
|        Justin Lin|    5|
|     Antoine Fuqua|    5|
+------------------+-----+
only showing top 10 rows



In [27]:
#filtering -- another transformation
condition1 = (df2.runtime.isNotNull()) |  (df2.Title.isNotNull())
df3 = df2.filter(condition1)
df3.show(5)

+----+--------------------+--------------------+--------------------+--------------------+--------------------+----+-------+------+------+-------+---------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|Year|runtime|Rating| Votes|revenue|Metascore|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+----+-------+------+------+-------+---------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|2014|    121|   8.1|757074| 333.13|     76.0|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|2012|    124|     7|485820| 126.46|     65.0|
|   3|               Split|     Horror,Thriller|Three girls are k...|  M. Night Shyamalan|James McAvoy, Any...|2016|    117|   7.3|157606| 138.12|     62.0|
|   4|                Sing|Animation,Comedy,...|In a city 

In [28]:
type(df3)

pyspark.sql.dataframe.DataFrame

In [33]:
df4=df3.filter(df3.Title.contains('Guardians'))
df4.show(5)

+----+--------------------+--------------------+--------------------+----------+--------------------+----+-------+------+------+-------+---------+
|Rank|               Title|               Genre|         Description|  Director|              Actors|Year|runtime|Rating| Votes|revenue|Metascore|
+----+--------------------+--------------------+--------------------+----------+--------------------+----+-------+------+------+-------+---------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|James Gunn|Chris Pratt, Vin ...|2014|    121|   8.1|757074| 333.13|     76.0|
+----+--------------------+--------------------+--------------------+----------+--------------------+----+-------+------+------+-------+---------+



In [34]:
#Zip an RDD with its element indices.
#https://amplab-extras.github.io/SparkR-pkg/rdocs/1.2/zipWithIndex.html
rdd=df2.rdd.zipWithIndex()
print(rdd.take(2))

[(Row(Rank=1, Title='Guardians of the Galaxy', Genre='Action,Adventure,Sci-Fi', Description='A group of intergalactic criminals are forced to work together to stop a fanatical warrior from taking control of the universe.', Director='James Gunn', Actors='Chris Pratt, Vin Diesel, Bradley Cooper, Zoe Saldana', Year='2014', runtime='121', Rating='8.1', Votes='757074', revenue=333.13, Metascore=76.0), 0), (Row(Rank=2, Title='Prometheus', Genre='Adventure,Mystery,Sci-Fi', Description='Following clues to the origin of mankind, a team finds a structure on a distant moon, but they soon realize they are not alone.', Director='Ridley Scott', Actors='Noomi Rapace, Logan Marshall-Green, Michael Fassbender, Charlize Theron', Year='2012', runtime='124', Rating='7', Votes='485820', revenue=126.46, Metascore=65.0), 1)]


In [35]:
print('RDD Count:', rdd.count())
print('RDD Collect as map:',  rdd.collectAsMap())
print('RDD Num Partitions:', rdd.getNumPartitions())

RDD Count: 1000
RDD Collect as map: {Row(Rank=1, Title='Guardians of the Galaxy', Genre='Action,Adventure,Sci-Fi', Description='A group of intergalactic criminals are forced to work together to stop a fanatical warrior from taking control of the universe.', Director='James Gunn', Actors='Chris Pratt, Vin Diesel, Bradley Cooper, Zoe Saldana', Year='2014', runtime='121', Rating='8.1', Votes='757074', revenue=333.13, Metascore=76.0): 0, Row(Rank=2, Title='Prometheus', Genre='Adventure,Mystery,Sci-Fi', Description='Following clues to the origin of mankind, a team finds a structure on a distant moon, but they soon realize they are not alone.', Director='Ridley Scott', Actors='Noomi Rapace, Logan Marshall-Green, Michael Fassbender, Charlize Theron', Year='2012', runtime='124', Rating='7', Votes='485820', revenue=126.46, Metascore=65.0): 1, Row(Rank=3, Title='Split', Genre='Horror,Thriller', Description='Three girls are kidnapped by a man with a diagnosed 23 distinct personalities. They must 